In [2]:
# Force the local gqcpy to be imported.
import sys
sys.path.insert(0, '../../build/gqcpy')

# Import the local gqcpy.
import gqcpy

# Import Python modules.
import itertools as it
import numpy as np

In [3]:
A = gqcpy.AdjacencyMatrix.Cyclic(6)
A.matrix()

array([[0, 1, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1],
       [1, 0, 0, 0, 1, 0]], dtype=uint64)

In [4]:
t = 1
U = 5

H = gqcpy.HoppingMatrix(A, t, U)
H.matrix()

array([[ 5., -1.,  0.,  0.,  0., -1.],
       [-1.,  5., -1.,  0.,  0.,  0.],
       [ 0., -1.,  5., -1.,  0.,  0.],
       [ 0.,  0., -1.,  5., -1.,  0.],
       [ 0.,  0.,  0., -1.,  5., -1.],
       [-1.,  0.,  0.,  0., -1.,  5.]])

In [5]:
hubbard_hamiltonian = gqcpy.HubbardHamiltonian(H)

In [6]:
K = 6  # number of sites
N_P = 3  # number of electron pairs

onv_basis = gqcpy.SpinResolvedONVBasis(K, N_P, N_P)

In [7]:
solver = gqcpy.EigenproblemSolver.Dense()
environment = gqcpy.CIEnvironment.Dense(hubbard_hamiltonian, onv_basis)

wfn = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateParameters()
wfn.coefficients().shape

(400,)

In [8]:
domain_partitions = np.array(list(it.product(range(2), repeat=6)))
domain_probabilities = np.zeros(domain_partitions.shape[0])

In [29]:
# We didn't take the "single site flip" criterion into account yet.
def MPD(ONV_a, Ia, ONV_b, Ib):
    ONV_a, ONV_b = np.array(list(repr(ONV_a))), np.array(list(repr(ONV_b)))
    
    domain = np.where(domain_partition == 1)[0]
    complement = np.where(domain_partition == 0)[0]
    
    Na, Nb = np.sum(ONV_a[domain].astype(np.float)), np.sum(ONV_b[domain].astype(np.float))
    
    
    if Na==1 and Nb==1:
        address = onv_basis.compoundAddress(Ia, Ib)
        domain_probabilities[d] += wfn.coefficients()[address]**2
        print(domain_partition, ONV_a, ONV_b, Na, Nb, wfn.coefficients()[address]**2)

In [30]:
for d,domain_partition in enumerate(domain_partitions):
    onv_basis.forEach(MPD)
    print(domain_probabilities[d])

0.0
[0 0 0 0 0 1] ['0' '0' '0' '1' '1' '1'] ['0' '0' '0' '1' '1' '1'] 1.0 1.0 1.48452400826018e-06
[0 0 0 0 0 1] ['0' '0' '0' '1' '1' '1'] ['0' '0' '1' '0' '1' '1'] 1.0 1.0 3.0355281167751794e-05
[0 0 0 0 0 1] ['0' '0' '0' '1' '1' '1'] ['0' '0' '1' '1' '0' '1'] 1.0 1.0 6.944504310130778e-05
[0 0 0 0 0 1] ['0' '0' '0' '1' '1' '1'] ['0' '1' '0' '0' '1' '1'] 1.0 1.0 6.944504310131039e-05
[0 0 0 0 0 1] ['0' '0' '0' '1' '1' '1'] ['0' '1' '0' '1' '0' '1'] 1.0 1.0 0.0004849524232858432
[0 0 0 0 0 1] ['0' '0' '0' '1' '1' '1'] ['0' '1' '1' '0' '0' '1'] 1.0 1.0 0.001235581607630222
[0 0 0 0 0 1] ['0' '0' '0' '1' '1' '1'] ['1' '0' '0' '0' '1' '1'] 1.0 1.0 6.486853996747974e-06
[0 0 0 0 0 1] ['0' '0' '0' '1' '1' '1'] ['1' '0' '0' '1' '0' '1'] 1.0 1.0 6.944504310130932e-05
[0 0 0 0 0 1] ['0' '0' '0' '1' '1' '1'] ['1' '0' '1' '0' '0' '1'] 1.0 1.0 0.0003167802948939634
[0 0 0 0 0 1] ['0' '0' '0' '1' '1' '1'] ['1' '1' '0' '0' '0' '1'] 1.0 1.0 7.748673269118932e-05
[0 0 0 0 0 1] ['0' '0' '1' '0' '1' '1

[0 0 0 0 1 1] ['0' '1' '1' '0' '0' '1'] ['1' '0' '1' '0' '0' '1'] 1.0 1.0 0.0006549298007628179
[0 0 0 0 1 1] ['0' '1' '1' '0' '0' '1'] ['1' '0' '1' '0' '1' '0'] 1.0 1.0 0.008517217727973916
[0 0 0 0 1 1] ['0' '1' '1' '0' '0' '1'] ['1' '1' '0' '0' '0' '1'] 1.0 1.0 6.944504310130742e-05
[0 0 0 0 1 1] ['0' '1' '1' '0' '0' '1'] ['1' '1' '0' '0' '1' '0'] 1.0 1.0 0.0012671211795758604
[0 0 0 0 1 1] ['0' '1' '1' '0' '1' '0'] ['0' '0' '1' '1' '0' '1'] 1.0 1.0 0.0012671211795758628
[0 0 0 0 1 1] ['0' '1' '1' '0' '1' '0'] ['0' '0' '1' '1' '1' '0'] 1.0 1.0 6.944504310130785e-05
[0 0 0 0 1 1] ['0' '1' '1' '0' '1' '0'] ['0' '1' '0' '1' '0' '1'] 1.0 1.0 0.008517217727973928
[0 0 0 0 1 1] ['0' '1' '1' '0' '1' '0'] ['0' '1' '0' '1' '1' '0'] 1.0 1.0 0.0006549298007628094
[0 0 0 0 1 1] ['0' '1' '1' '0' '1' '0'] ['0' '1' '1' '0' '0' '1'] 1.0 1.0 0.0006549298007628134
[0 0 0 0 1 1] ['0' '1' '1' '0' '1' '0'] ['0' '1' '1' '0' '1' '0'] 1.0 1.0 9.423707531528463e-05
[0 0 0 0 1 1] ['0' '1' '1' '0' '1' '0'] ['

[0 0 0 1 0 0] ['1' '0' '0' '1' '1' '0'] ['1' '0' '1' '1' '0' '0'] 1.0 1.0 0.0002777801724052306
[0 0 0 1 0 0] ['1' '0' '0' '1' '1' '0'] ['1' '1' '0' '1' '0' '0'] 1.0 1.0 0.000603614418293866
[0 0 0 1 0 0] ['1' '0' '1' '1' '0' '0'] ['0' '0' '0' '1' '1' '1'] 1.0 1.0 0.0003167802948939623
[0 0 0 1 0 0] ['1' '0' '1' '1' '0' '0'] ['0' '0' '1' '1' '0' '1'] 1.0 1.0 0.0006549298007628101
[0 0 0 1 0 0] ['1' '0' '1' '1' '0' '0'] ['0' '0' '1' '1' '1' '0'] 1.0 1.0 6.944504310130765e-05
[0 0 0 1 0 0] ['1' '0' '1' '1' '0' '0'] ['0' '1' '0' '1' '0' '1'] 1.0 1.0 0.008517217727973925
[0 0 0 1 0 0] ['1' '0' '1' '1' '0' '0'] ['0' '1' '0' '1' '1' '0'] 1.0 1.0 0.0012671211795758635
[0 0 0 1 0 0] ['1' '0' '1' '1' '0' '0'] ['0' '1' '1' '1' '0' '0'] 1.0 1.0 3.0355281167754823e-05
[0 0 0 1 0 0] ['1' '0' '1' '1' '0' '0'] ['1' '0' '0' '1' '0' '1'] 1.0 1.0 0.0006036144182938692
[0 0 0 1 0 0] ['1' '0' '1' '1' '0' '0'] ['1' '0' '0' '1' '1' '0'] 1.0 1.0 0.0002777801724052306
[0 0 0 1 0 0] ['1' '0' '1' '1' '0' '0'] [

[0 0 0 1 1 0] ['0' '0' '1' '0' '1' '1'] ['0' '0' '1' '1' '0' '1'] 1.0 1.0 0.0006549298007628037
[0 0 0 1 1 0] ['0' '0' '1' '0' '1' '1'] ['0' '1' '0' '0' '1' '1'] 1.0 1.0 0.0006549298007628026
[0 0 0 1 1 0] ['0' '0' '1' '0' '1' '1'] ['0' '1' '0' '1' '0' '1'] 1.0 1.0 0.008517217727973904
[0 0 0 1 1 0] ['0' '0' '1' '0' '1' '1'] ['0' '1' '1' '0' '1' '0'] 1.0 1.0 0.0002777801724052308
[0 0 0 1 1 0] ['0' '0' '1' '0' '1' '1'] ['0' '1' '1' '1' '0' '0'] 1.0 1.0 0.00031678029489396397
[0 0 0 1 1 0] ['0' '0' '1' '0' '1' '1'] ['1' '0' '0' '0' '1' '1'] 1.0 1.0 6.944504310130761e-05
[0 0 0 1 1 0] ['0' '0' '1' '0' '1' '1'] ['1' '0' '0' '1' '0' '1'] 1.0 1.0 0.001267121179575858
[0 0 0 1 1 0] ['0' '0' '1' '0' '1' '1'] ['1' '0' '1' '0' '1' '0'] 1.0 1.0 0.0009672820907607613
[0 0 0 1 1 0] ['0' '0' '1' '0' '1' '1'] ['1' '0' '1' '1' '0' '0'] 1.0 1.0 0.003264743363938498
[0 0 0 1 1 0] ['0' '0' '1' '0' '1' '1'] ['1' '1' '0' '0' '1' '0'] 1.0 1.0 0.0032647433639384966
[0 0 0 1 1 0] ['0' '0' '1' '0' '1' '1'] ['

[0 0 0 1 1 1] ['1' '1' '0' '0' '1' '0'] ['0' '1' '1' '0' '0' '1'] 1.0 1.0 0.0012671211795758663
[0 0 0 1 1 1] ['1' '1' '0' '0' '1' '0'] ['0' '1' '1' '0' '1' '0'] 1.0 1.0 0.00027778017240523184
[0 0 0 1 1 1] ['1' '1' '0' '0' '1' '0'] ['0' '1' '1' '1' '0' '0'] 1.0 1.0 0.0003167802948939654
[0 0 0 1 1 1] ['1' '1' '0' '0' '1' '0'] ['1' '0' '1' '0' '0' '1'] 1.0 1.0 0.0007106881166866412
[0 0 0 1 1 1] ['1' '1' '0' '0' '1' '0'] ['1' '0' '1' '0' '1' '0'] 1.0 1.0 0.0009672820907607744
[0 0 0 1 1 1] ['1' '1' '0' '0' '1' '0'] ['1' '0' '1' '1' '0' '0'] 1.0 1.0 0.0032647433639384996
[0 0 0 1 1 1] ['1' '1' '0' '0' '1' '0'] ['1' '1' '0' '0' '0' '1'] 1.0 1.0 3.0355281167754728e-05
[0 0 0 1 1 1] ['1' '1' '0' '0' '1' '0'] ['1' '1' '0' '0' '1' '0'] 1.0 1.0 9.423707531528558e-05
[0 0 0 1 1 1] ['1' '1' '0' '0' '1' '0'] ['1' '1' '0' '1' '0' '0'] 1.0 1.0 0.0006549298007628117
[0 0 0 1 1 1] ['1' '1' '0' '1' '0' '0'] ['0' '1' '1' '0' '0' '1'] 1.0 1.0 0.0007106881166866399
[0 0 0 1 1 1] ['1' '1' '0' '1' '0' '0'

...

0.08381666106356
[1 1 0 0 0 0] ['0' '1' '0' '0' '1' '1'] ['0' '1' '0' '0' '1' '1'] 1.0 1.0 9.42370753152906e-05
[1 1 0 0 0 0] ['0' '1' '0' '0' '1' '1'] ['0' '1' '0' '1' '0' '1'] 1.0 1.0 0.0009672820907607722
[1 1 0 0 0 0] ['0' '1' '0' '0' '1' '1'] ['0' '1' '0' '1' '1' '0'] 1.0 1.0 0.0002777801724052299
[1 1 0 0 0 0] ['0' '1' '0' '0' '1' '1'] ['0' '1' '1' '0' '0' '1'] 1.0 1.0 0.0002777801724052303
[1 1 0 0 0 0] ['0' '1' '0' '0' '1' '1'] ['0' '1' '1' '0' '1' '0'] 1.0 1.0 0.0006036144182938677
[1 1 0 0 0 0] ['0' '1' '0' '0' '1' '1'] ['0' '1' '1' '1' '0' '0'] 1.0 1.0 0.0012355816076302134
[1 1 0 0 0 0] ['0' '1' '0' '0' '1' '1'] ['1' '0' '0' '0' '1' '1'] 1.0 1.0 3.0355281167754328e-05
[1 1 0 0 0 0] ['0' '1' '0' '0' '1' '1'] ['1' '0' '0' '1' '0' '1'] 1.0 1.0 0.0007106881166866389
[1 1 0 0 0 0] ['0' '1' '0' '0' '1' '1'] ['1' '0' '0' '1' '1' '0'] 1.0 1.0 0.0012671211795758565
[1 1 0 0 0 0] ['0' '1' '0' '0' '1' '1'] ['1' '0' '1' '0' '0' '1'] 1.0 1.0 0.0012671211795758643
[1 1 0 0 0 0] ['0' '1' 

[1 1 0 0 0 1] ['0' '1' '0' '1' '1' '0'] ['0' '0' '0' '1' '1' '1'] 1.0 1.0 6.944504310130929e-05
[1 1 0 0 0 1] ['0' '1' '0' '1' '1' '0'] ['0' '0' '1' '0' '1' '1'] 1.0 1.0 0.0012671211795758559
[1 1 0 0 0 1] ['0' '1' '0' '1' '1' '0'] ['0' '0' '1' '1' '0' '1'] 1.0 1.0 0.0007106881166866376
[1 1 0 0 0 1] ['0' '1' '0' '1' '1' '0'] ['0' '1' '0' '1' '1' '0'] 1.0 1.0 9.423707531528921e-05
[1 1 0 0 0 1] ['0' '1' '0' '1' '1' '0'] ['0' '1' '1' '0' '1' '0'] 1.0 1.0 0.0006549298007628095
[1 1 0 0 0 1] ['0' '1' '0' '1' '1' '0'] ['0' '1' '1' '1' '0' '0'] 1.0 1.0 6.944504310130768e-05
[1 1 0 0 0 1] ['0' '1' '0' '1' '1' '0'] ['1' '0' '0' '1' '1' '0'] 1.0 1.0 0.0006549298007628025
[1 1 0 0 0 1] ['0' '1' '0' '1' '1' '0'] ['1' '0' '1' '0' '1' '0'] 1.0 1.0 0.008517217727973932
[1 1 0 0 0 1] ['0' '1' '0' '1' '1' '0'] ['1' '0' '1' '1' '0' '0'] 1.0 1.0 0.0012671211795758609
[1 1 0 0 0 1] ['0' '1' '1' '0' '1' '0'] ['0' '0' '0' '1' '1' '1'] 1.0 1.0 0.0003167802948939629
[1 1 0 0 0 1] ['0' '1' '1' '0' '1' '0'] [

[1 1 0 0 1 1] ['1' '0' '1' '1' '0' '0'] ['0' '0' '1' '1' '0' '1'] 1.0 1.0 0.0006549298007628101
[1 1 0 0 1 1] ['1' '0' '1' '1' '0' '0'] ['0' '0' '1' '1' '1' '0'] 1.0 1.0 6.944504310130765e-05
[1 1 0 0 1 1] ['1' '0' '1' '1' '0' '0'] ['0' '1' '1' '1' '0' '0'] 1.0 1.0 3.0355281167754823e-05
[1 1 0 0 1 1] ['1' '0' '1' '1' '0' '0'] ['1' '0' '1' '1' '0' '0'] 1.0 1.0 9.423707531528484e-05
0.7034646664865373
[1 1 0 1 0 0] ['0' '0' '0' '1' '1' '1'] ['0' '0' '0' '1' '1' '1'] 1.0 1.0 1.48452400826018e-06
[1 1 0 1 0 0] ['0' '0' '0' '1' '1' '1'] ['0' '0' '1' '1' '0' '1'] 1.0 1.0 6.944504310130778e-05
[1 1 0 1 0 0] ['0' '0' '0' '1' '1' '1'] ['0' '0' '1' '1' '1' '0'] 1.0 1.0 6.486853996747512e-06
[1 1 0 1 0 0] ['0' '0' '0' '1' '1' '1'] ['0' '1' '0' '0' '1' '1'] 1.0 1.0 6.944504310131039e-05
[1 1 0 1 0 0] ['0' '0' '0' '1' '1' '1'] ['0' '1' '1' '0' '0' '1'] 1.0 1.0 0.001235581607630222
[1 1 0 1 0 0] ['0' '0' '0' '1' '1' '1'] ['0' '1' '1' '0' '1' '0'] 1.0 1.0 0.00031678029489396706
[1 1 0 1 0 0] ['0' '0

[1 1 1 0 0 0] ['0' '0' '1' '0' '1' '1'] ['0' '1' '0' '1' '0' '1'] 1.0 1.0 0.008517217727973904
[1 1 1 0 0 0] ['0' '0' '1' '0' '1' '1'] ['0' '1' '0' '1' '1' '0'] 1.0 1.0 0.001267121179575857
[1 1 1 0 0 0] ['0' '0' '1' '0' '1' '1'] ['1' '0' '0' '0' '1' '1'] 1.0 1.0 6.944504310130761e-05
[1 1 1 0 0 0] ['0' '0' '1' '0' '1' '1'] ['1' '0' '0' '1' '0' '1'] 1.0 1.0 0.001267121179575858
[1 1 1 0 0 0] ['0' '0' '1' '0' '1' '1'] ['1' '0' '0' '1' '1' '0'] 1.0 1.0 0.0007106881166866393
[1 1 1 0 0 0] ['0' '0' '1' '1' '0' '1'] ['0' '0' '1' '0' '1' '1'] 1.0 1.0 0.000654929800762805
[1 1 1 0 0 0] ['0' '0' '1' '1' '0' '1'] ['0' '0' '1' '1' '0' '1'] 1.0 1.0 9.423707531528965e-05
[1 1 1 0 0 0] ['0' '0' '1' '1' '0' '1'] ['0' '0' '1' '1' '1' '0'] 1.0 1.0 3.0355281167754203e-05
[1 1 1 0 0 0] ['0' '0' '1' '1' '0' '1'] ['0' '1' '0' '0' '1' '1'] 1.0 1.0 0.0032647433639384988
[1 1 1 0 0 0] ['0' '0' '1' '1' '0' '1'] ['0' '1' '0' '1' '0' '1'] 1.0 1.0 0.0009672820907607715
[1 1 1 0 0 0] ['0' '0' '1' '1' '0' '1'] ['0

0.0838166610635599
0.08381666106355992
0.0


In [19]:
domain_probabilities


array([0.        , 0.08381666, 0.08381666, 0.69389728, 0.08381666,
       0.28633257, 0.69389728, 0.16274974, 0.08381666, 0.48363378,
       0.28633257, 0.18959364, 0.69389728, 0.18959364, 0.16274974,
       0.69389728, 0.08381666, 0.28633257, 0.48363378, 0.18959364,
       0.28633257, 0.12860574, 0.18959364, 0.28633257, 0.69389728,
       0.18959364, 0.18959364, 0.48363378, 0.16274974, 0.28633257,
       0.69389728, 0.08381666, 0.08381666, 0.69389728, 0.28633257,
       0.16274974, 0.48363378, 0.18959364, 0.18959364, 0.69389728,
       0.28633257, 0.18959364, 0.12860574, 0.28633257, 0.18959364,
       0.48363378, 0.28633257, 0.08381666, 0.69389728, 0.16274974,
       0.18959364, 0.69389728, 0.18959364, 0.28633257, 0.48363378,
       0.08381666, 0.16274974, 0.69389728, 0.28633257, 0.08381666,
       0.69389728, 0.08381666, 0.08381666, 0.        ])

In [20]:
for dpart, dprob in zip(domain_partitions, domain_probabilities):
    print(dpart, dprob)

[0 0 0 0 0 0] 0.0
[0 0 0 0 0 1] 0.08381666106355992
[0 0 0 0 1 0] 0.0838166610635599
[0 0 0 0 1 1] 0.6938972774603235
[0 0 0 1 0 0] 0.08381666106355999
[0 0 0 1 0 1] 0.28633257408893886
[0 0 0 1 1 0] 0.6938972774603225
[0 0 0 1 1 1] 0.16274974125160746
[0 0 1 0 0 0] 0.08381666106355992
[0 0 1 0 0 1] 0.48363378063669066
[0 0 1 0 1 0] 0.28633257408893903
[0 0 1 0 1 1] 0.18959363576091767
[0 0 1 1 0 0] 0.6938972774603231
[0 0 1 1 0 1] 0.18959363576091756
[0 0 1 1 1 0] 0.16274974125160754
[0 0 1 1 1 1] 0.6938972774603231
[0 1 0 0 0 0] 0.08381666106356
[0 1 0 0 0 1] 0.2863325740889389
[0 1 0 0 1 0] 0.4836337806366902
[0 1 0 0 1 1] 0.18959363576091756
[0 1 0 1 0 0] 0.28633257408893914
[0 1 0 1 0 1] 0.12860573576460424
[0 1 0 1 1 0] 0.18959363576091753
[0 1 0 1 1 1] 0.28633257408893886
[0 1 1 0 0 0] 0.6938972774603228
[0 1 1 0 0 1] 0.18959363576091748
[0 1 1 0 1 0] 0.1895936357609176
[0 1 1 0 1 1] 0.48363378063669044
[0 1 1 1 0 0] 0.16274974125160752
[0 1 1 1 0 1] 0.28633257408893886
[0 1 1 1